# First steps with COSIPY

This is the first tutorial to get you started with using COSIPY. In it we will go through how to

* Setup a simulation for a single gridpoint
* Run the simulation and save the results
* Do simple visualisations of the results using matplotlib

First we have to import the python libraries we are going to use

In [ ]:
# Have to change the cwd for the ipython session, otherwise COSIPY
# will look for things in the wrong places.
import os
# This is not really a good method, if cell is re run we end up in the
# wrong directory.
os.chdir('./../')

In [ ]:
from cosipy.cpkernel.cosipy_core import cosipy_core
from cosipy.cpkernel.io import IOClass
# We need io for changing config parameters/constants.
import cosipy.cpkernel.io as io
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Have to tell matplotlib to plot inline
%matplotlib inline

# Create input/output files
First we have to creat input/ouput files for the simulation. This is handled by the `IOClass` of COSIPY which creates an xarray dataset containing information about the spatial and temporal coordinate, the data varialbes and the attributes used in the simulation. 

Before we initialize these files, it is interesting to have a look at the varialbes and constants used by the `io` module and the rest of the model, they  are contained in a dictionary called `NAMELIST` which is available from the `cfg` module:

In [ ]:
import cfg
NAMELIST = cfg.NAMELIST
NAMELIST

These are loaded from the files `config.cfg` and `constants.cfg` located in the root directory of cosipy. We can change any of these variables to customise our simulation. This can by done changing the value of a variable in the previously mentioned files, however for this to take affect we have to reload the `cfg` moduel. 

A much simpler way is to change it directly in the `NAMELIST` before we begin initializing our run, directly in this session. For instance we can change the variable `time_end` to adjust the temporal extent of our simulation:

In [ ]:
# First take a look at the default value
NAMELIST['time_end']

In [ ]:
# Then we change it
NAMELIST['time_end'] = '2009-01-08T00:00'

We are now ready to initialise the `IOClass`, we have to pass the `NAMELIST` to it:

In [ ]:
# Initialize the IOClass
IO = IOClass(NAMELIST)

With the `IOClass` ready, we can initialise the data files needed for our run:

In [ ]:
# Create files for the data, results and restart.
DATA = IO.create_data_file()
RESULTS = IO.create_result_file()
RESTART = IO.create_restart_file()

The output above gives us some helpful insight about the data. We can see that the data covers the period 2009-01-01 to 2009-01-31, and that we have specified the integration time to 2009-01-01 to 2009-01-08 by changing `io.time_end`. We also see that we have one gridpoint which we will perform the calculation on.

It also shows some quality control on the input data, more on this later.

Note that we don't have to change `NAMELIST['time_end']` as we did above - if we don't, the simulation will run using the default parameters from `config.cfg` and `constants.cfg`.

Also note that the `IOClass` along with the files DATA, RESULTS and RESTART has to be re-initliazed after a variable has been changed in `NAMELIST`.

# Running COSIPY
Now we are ready to do a simple point simulation with COSIPY. This example is run on the Zhadang glacier. We can change this by changing the `input_netcdf` in `NAMELIST` to point at the files we want to use. In these tutorials data for Zhadang and Hintereisferner are included.

For this run, we are not going to evaluate the data using ablation stakes, hence we set the variables

In [ ]:
stakes_loc = None
df_stakes_data = None
stake_names = None

We can then run the model by simply calling the `cosipy_core` function and passing the `NAMELIST` to it:

*Note that this can take some time*

In [ ]:
# We pass the index of our point to cosipy_core, since python is zero
# indexed we have to subtract one.
x = 0
y = 0
model = cosipy_core(DATA.isel(lat=y, lon=x), y, x, NAMELIST, stake_names=stake_names,
                    stake_data=df_stakes_data)
# Create numpy arrays which aggregates all local results
IO.create_global_result_arrays()


# Here we are unpacking the results from the model run, getting ready to save
# it to our RESULTS dataframe.
indY,indX,local_restart,RAIN,SNOWFALL,LWin,LWout,H,LE,B,QRR,MB,surfMB,Q,SNOWHEIGHT,TOTALHEIGHT,TS,ALBEDO,NLAYERS, \
                ME,intMB,EVAPORATION,SUBLIMATION,CONDENSATION,DEPOSITION,REFREEZE,subM,Z0,surfM,MOL, \
                LAYER_HEIGHT,LAYER_RHO,LAYER_T,LAYER_LWC,LAYER_CC,LAYER_POROSITY,LAYER_ICE_FRACTION, \
                LAYER_IRREDUCIBLE_WATER,LAYER_REFREEZE,stake_names,stat,df_eval = model

IO.copy_local_to_global(indY,indX,RAIN,SNOWFALL,LWin,LWout,H,LE,B,QRR,MB,surfMB,Q,SNOWHEIGHT,TOTALHEIGHT,TS,ALBEDO,NLAYERS, \
                ME,intMB,EVAPORATION,SUBLIMATION,CONDENSATION,DEPOSITION,REFREEZE,subM,Z0,surfM,MOL,LAYER_HEIGHT,LAYER_RHO, \
                LAYER_T,LAYER_LWC,LAYER_CC,LAYER_POROSITY,LAYER_ICE_FRACTION,LAYER_IRREDUCIBLE_WATER,LAYER_REFREEZE)

# Write results to file
IO.write_results_to_file()


Thats it! Now we are ready to inspect the results.

# Visualizing the results
To get a quick overview of the variables in our dataframe, we can simply call it 

In [ ]:
RESULTS

We can see that our data is stored in an xarray dataset - a very handy format for interacting with netcdf files in python. To check which data variables are available, one can simply click on the label "Data Variables", which will expand a list of the variables.

We can quckly plot a variable, the surface mass balance (`surfMB`) for instance:

In [ ]:
RESULTS.surfMB.plot();

As you can see, the use of xarray-datasets for storing the data makes it possible to levarage the built in plotting functionality of xarray and quickly visualize the output.

For a slightly more advanced plot, one can still make use of matplotlib

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(7,8), nrows=2, sharex=True,
                               gridspec_kw = {'height_ratios':[2,1]})
RESULTS.surfMB.plot(ax=ax1)
RESULTS.SNOWFALL.plot(ax=ax2, color='C1')
# Easier to make use of xarray plotting capabilites and then remove unwanted lables.
ax1.set_xlabel('')
ax2.set_title('')
# Get the time span of our dataset to use for title
time_span = RESULTS.time[0].values, RESULTS.time[-1].values
time_span = np.datetime_as_string(time_span, unit='m')
# Set the title of the plot.
ax1.set_title(f'Surface mass balance and Snowfall at Zhadang \n between {time_span[0]} and' +\
              f' {time_span[1]}')
# Fixes the spacing between parts of the plots.
fig.tight_layout()

As we can see in the figures above, the positive surface mass balances during the 6th or January can be explained, as one might expect, by snowfall.

## Next steps
[Back to overview](welcome.ipynb)

[Sensitivty studies with COSIPY](sensitivty_study.ipynb)